# Deep Neural Networks Laboration

Data used in this laboration are from the Kitsune Network Attack Dataset, https://archive.ics.uci.edu/ml/datasets/Kitsune+Network+Attack+Dataset . We will focus on the 'Mirai' part of the dataset. Your task is to make a DNN that can classify if each attack is benign or malicious. The dataset has 116 covariates, but to make it a bit more difficult we will remove the first 24 covariates.

You need to answer all questions in this notebook.

# Part 1: Get the data

Skip this part if you load stored numpy arrays (Mirai*.npy) (which is recommended)

Use `wget` in the terminal of your cloud machine (in the same directory as where you have saved this notebook) to download the data, i.e.

wget https://archive.ics.uci.edu/ml/machine-learning-databases/00516/mirai/Mirai_dataset.csv.gz

wget https://archive.ics.uci.edu/ml/machine-learning-databases/00516/mirai/Mirai_labels.csv.gz

Then unpack the files using `gunzip` in the terminal, i.e.

gunzip Mirai_dataset.csv.gz

gunzip Mirai_labels.csv.gz

# Part 2: Get a graphics card

Skip this part if you run on the CPU (recommended)

Lets make sure that our script can see the graphics card that will be used. The graphics cards will perform all the time consuming calculations in every training iteration.

In [3]:
import os
import warnings

# Ignore FutureWarning from numpy
warnings.simplefilter(action='ignore', category=FutureWarning)

import keras.backend as K
import tensorflow as tf

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Allow growth of GPU memory, otherwise it will always look like all the memory is being used
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True) 

ModuleNotFoundError: No module named 'keras'

# Part 3: Hardware

In deep learning, the computer hardware is very important. You should always know what kind of hardware you are working on. Lets pretend that everyone is using an Nvidia RTX 3090 graphics card.

Question 1: Google the name of the graphics card, how many CUDA cores does it have?

Question 2: How much memory does the graphics card have?

Question 3: What is stored in the GPU memory while training a DNN ?

Q1: 10 496

Q2: 24 GB

Q3:     
      1. Parameters — The weights and biases of the network.
      2. Optimizer’s variables — Per-algorithm intermediate variables (e.g. momentums).
      3. Intermediate calculations — Values from the forward pass that are temporarily stored in GPU memory and then used in the backward pass. (e.g. the activation outputs of every layer are used in the backward pass to calculate the gradients)
      4. Workspace — Temporary memory for local variables of kernel implementations.

While (1) and (4) are always required, (2) and (3) are required only in training mode.

# Part 4: Load the data

To make this step easier, directly load the data from saved numpy arrays (.npy) (recommended)


Load the dataset from the csv files, it will take some time since it is almost 1.4 GB. (not recommended, unless you want to learn how to do it)

We will use the function `genfromtxt` to load the data. (not recommended, unless you want to learn how to do it)

https://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html

Load the data from csv files the first time, then save the data as numpy files for faster loading the next time.


In [1]:
from numpy import genfromtxt # Not needed if you load data from numpy arrays
import numpy as np

# Load data from numpy arrays, choose reduced files if the training takes too long
X = np.load('Mirai_data.npy')
Y = np.load('Mirai_labels.npy')


# Remove the first 24 covariates (columns)
X = X[:, 24:]

print('The covariates have size {}.'.format(X.shape))
print('The labels have size {}.'.format(Y.shape))

# Print the number of examples of each class
unique, counts = np.unique(Y, return_counts=True)
dict(zip(unique, counts))

The covariates have size (764137, 92).
The labels have size (764137,).


{0.0: 121621, 1.0: 642516}

# Part 5: How good is a naive classifier?

Question 4: Given the number of examples from each class, how high classification performance can a naive classifier obtain? The naive classifier will assume that all examples belong to one class. Note: you do not need to make a naive classifier, this is a theoretical question, just to understand how good performance we can obtain by guessing that all examples belong to one class.

Q4: if p = examples of a class in a set n, then a naive classifier can achieve classification performance p/n

In [5]:
# It is common to have NaNs in the data, lets check for it. Hint: np.isnan()

# Print the number of NaNs (not a number) in the labels
clean_Y = np.count_nonzero(~np.isnan(Y))
print(clean_Y)

# Print the number of NaNs in the covariates
clean_X = np.count_nonzero(~np.isnan(X))
print(clean_X)

764137
70300604


# Part 6: Preprocessing

Lets do some simple preprocessing

In [2]:
# Convert covariates to floats
X = X.astype(float)

# Convert labels to integers
Y = Y.astype(int)

# Remove mean of each covariate (column)
X = X - np.mean(X, axis = 0)

# Divide each covariate (column) by its standard deviation
X = X/ np.std(X, axis = 0)

# Check that mean is 0 and standard deviation is 1 for all covariates, by printing mean and std
print(np.mean(X))
print(np.std(X))

-7.309755941969869e-14
1.0000000000003428


# Part 7: Split the dataset

Use the first 70% of the dataset for training, leave the other 30% for validation and test, call the variables

Xtrain (70%)

Xtemp  (30%)

Ytrain (70%)

Ytemp  (30%)

We use a function from scikit learn.
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html


In [3]:
from sklearn.model_selection import train_test_split

# Your code

Xtrain, Xtemp, Ytrain, Ytemp = train_test_split(X, Y, train_size=0.70, random_state=42)


#Xtrain, Xtemp = train_test_split(X, train_size = 0.70, random_state=42)
#Ytrain, Ytemp = train_test_split(Y, train_size = 0.70, random_state=42)

print('Xtrain has size {}.'.format(Xtrain.shape))
print('Ytrain has size {}.'.format(Ytrain.shape))

print('Xtemp has size {}.'.format(Xtemp.shape))
print('Ytemp has size {}.'.format(Ytemp.shape))

# Print the number of examples of each class, for the training data and the remaining 30%
print(np.bincount(Ytrain)[0], np.bincount(Ytrain)[1])
print(np.bincount(Ytemp)[0], np.bincount(Ytemp)[1])


Xtrain has size (534895, 92).
Ytrain has size (534895,).
Xtemp has size (229242, 92).
Ytemp has size (229242,).
85248 449647
36373 192869


#  Part 8: Split non-training data data into validation and test
Now split your non-training data (Xtemp, Ytemp) into 50% validation (Xval, Yval) and 50% testing (Xtest, Ytest), we use a function from scikit learn. In total this gives us 70% for training, 15% for validation, 15% for test.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Do all variables (Xtrain,Ytrain), (Xval,Yval), (Xtest,Ytest) have the shape that you expect?

In [4]:
from sklearn.model_selection import train_test_split


Xval, Xtest = train_test_split(Xtemp, train_size = 0.50, random_state=42)
Yval, Ytest = train_test_split(Ytemp, train_size = 0.50, random_state=42)

print('The validation and test data have size {}, {}, {} and {}'.format(Xval.shape, Xtest.shape, Yval.shape, Ytest.shape))

The validation and test data have size (114621, 92), (114621, 92), (114621,) and (114621,)


# Part 9: DNN classification

Finish this code to create a first version of the classifier using a DNN. Start with a simple network with 2 dense layers (with 20 nodes each), using sigmoid activation functions. The final dense layer should have a single node and a sigmoid activation function. We start with the SGD optimizer.

For different parts of this notebook you need to go back here, add more things, and re-run this cell to re-define the build function.

Relevant functions are

`model.add()`, adds a layer to the network

`Dense()`, a dense network layer

`model.compile()`, compile the model, add " metrics=['accuracy'] " to print the classification accuracy during the training

See https://keras.io/layers/core/ for information on how the `Dense()` function works

Import a relevant cost / loss function for binary classification from keras.losses (https://keras.io/losses/)

See the following links for how to compile, train and evaluate the model

https://keras.io/api/models/model_training_apis/#compile-method

https://keras.io/api/models/model_training_apis/#fit-method

https://keras.io/api/models/model_training_apis/#evaluate-method


In [5]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD
from keras.losses import BinaryCrossentropy

# Set seed from random number generator, for better comparisons
from numpy.random import seed
seed(123)

def build_DNN(input_shape, n_layers, n_nodes, act_fun='sigmoid', optimizer='sgd', learning_rate=0.01, 
              use_bn=False, use_dropout=False, use_custom_dropout=False):
    
    # Setup optimizer, depending on input parameter string
    opt = SGD(learning_rate = learning_rate)
    
    # Setup a sequential model
    model = Sequential()

    # Add layers to the model, using the input parameters of the build_DNN function
    
    
    # Add first layer, requires input shape
    model.add(Dense(units = n_nodes,
                    input_dim = input_shape, 
                    activation = act_fun))
    
    # Add remaining layers, do not require input shape
    for i in range(n_layers-1):
        model.add(Dense(units = n_nodes,
                       activation = act_fun))
           
    
    # Add final layer
    model.add(Dense(units=1, activation = act_fun)) # 1 node
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer = opt,
                  metrics=['accuracy'])
    
    return model

In [6]:
# Lets define a help function for plotting the training results

import matplotlib.pyplot as plt
def plot_results(history):
    
    val_loss = history.history['val_loss']
    acc = history.history['accuracy']
    loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    
    plt.figure(figsize=(10,4))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(loss)
    plt.plot(val_loss)
    plt.legend(['Training','Validation'])

    plt.figure(figsize=(10,4))
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(acc)
    plt.plot(val_acc)
    plt.legend(['Training','Validation'])

    plt.show()

# Part 10: Train the DNN

Time to train the DNN, we start simple with 2 layers with 20 nodes each, learning rate 0.1.

Relevant functions

`build_DNN`, the function we defined in Part 9, call it with the parameters you want to use

`model.fit()`, train the model with some training data

`model.evaluate()`, apply the trained model to some test data

See the following links for how to train and evaluate the model

https://keras.io/api/models/model_training_apis/#fit-method

https://keras.io/api/models/model_training_apis/#evaluate-method


### 2 layers, 20 nodes

In [23]:
# Setup some training parameters
batch_size = 10000
epochs = 20

input_shape = 92

# Build the model
model1 = build_DNN(input_shape, n_layers = 2, n_nodes = 20, learning_rate = 0.1)

# Train the model, provide training data and validation data
history1 = model1.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size)


Epoch 1/20
54/54 [==============================] - 1s 8ms/step - loss: 0.4178 - accuracy: 0.8406 - val_loss: 0.3955 - val_accuracy: 0.8404
Epoch 2/20
54/54 [==============================] - 0s 6ms/step - loss: 0.3718 - accuracy: 0.8406 - val_loss: 0.3443 - val_accuracy: 0.8404
Epoch 3/20
54/54 [==============================] - 0s 5ms/step - loss: 0.3167 - accuracy: 0.8406 - val_loss: 0.2872 - val_accuracy: 0.8404
Epoch 4/20
54/54 [==============================] - 0s 5ms/step - loss: 0.2651 - accuracy: 0.8447 - val_loss: 0.2435 - val_accuracy: 0.8614
Epoch 5/20
54/54 [==============================] - 0s 5ms/step - loss: 0.2312 - accuracy: 0.8699 - val_loss: 0.2185 - val_accuracy: 0.8797
Epoch 6/20
54/54 [==============================] - 0s 5ms/step - loss: 0.2127 - accuracy: 0.8813 - val_loss: 0.2051 - val_accuracy: 0.8863
Epoch 7/20
54/54 [==============================] - 0s 6ms/step - loss: 0.2025 - accuracy: 0.8926 - val_loss: 0.1973 - val_accuracy: 0.8992
Epoch 8/20
54/54 [==

In [ ]:
# Evaluate the model on the test data
score = model1.evaluate(Xtest, Ytest)

print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
# Plot the history from the training run
plot_results(history1)

# Part 11: More questions

Question 5: What happens if you add several Dense layers without specifying the activation function?

Question 6: How are the weights in each dense layer initialized as default? How are the bias weights initialized?


Q5: It will use linear activation, which in turn means adding several Dense layers is redundant.

Q6: Normaly distributed with stddev = 0.01. As zero.

# Part 12: Balancing the classes

This dataset is rather unbalanced, we need to define class weights so that the training pays more attention to the class with fewer samples. We use a function in scikit learn

https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html

You need to call the function something like this

class_weights = class_weight.compute_class_weight(class_weight = , classes = , y = )

otherwise it will complain

In [8]:
from sklearn.utils import class_weight

# Calculate class weights
class_weights = class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(Ytrain), y = Ytrain)

# Print the class weights
print(class_weights)
# Keras wants the weights in this form, uncomment and change value1 and value2 to your weights, 
# or get them from the array that is returned from class_weight

class_weights = {0: class_weights[0],
                 1: class_weights[1]}


[3.13728768 0.59479436]


### 2 layers, 20 nodes, class weights

In [ ]:
# Setup some training parameters
batch_size = 10000
epochs = 20
input_shape = 92

# Build and train model
model2 = build_DNN(input_shape, n_layers = 2, n_nodes = 20, learning_rate = 0.1)

# Train the model, provide training data and validation data
history2 = model2.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)

In [ ]:
# Evaluate model on test data
score = model2.evaluate(Xtest, Ytest)

print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
plot_results(history2)

# Part 13: More questions

Skip questions 8 and 9 if you run on the CPU (recommended)

Question 7: Why do we have to use a batch size? Why can't we simply use all data at once? This is more relevant for even larger datasets.

Question 8: How busy is the GPU for a batch size of 100? How much GPU memory is used? Hint: run 'nvidia-smi' on the computer a few times during training.

Question 9: What is the processing time for one training epoch when the batch size is 100? What is the processing time for one epoch when the batch size is 1,000? What is the processing time for one epoch when the batch size is 10,000? Explain the results. 

Question 10: How many times are the weights in the DNN updated in each training epoch if the batch size is 100? How many times are the weights in the DNN updated in each training epoch if the batch size is 1,000? How many times are the weights in the DNN updated in each training epoch if the batch size is 10,000?  

Question 11: What limits how large the batch size can be?

Question 12: Generally speaking, how is the learning rate related to the batch size? If the batch size is decreased, how should the learning rate be changed?

Lets use a batch size of 10,000 from now on, and a learning rate of 0.1.

Q7: We use batch sizes to split our data into more manageable sub-datasets. As we train our network on a batch instead of the entire dataset, it requires less memory which is important when we work with large dataset with a lot of features -> meaning we may not be able to train our network unless we divide it in some way.

Q10: The weights are updated after processing each batch. This means that if we use a data set of 10^5 samples and a batch size of 100, we will split our data set into 1000 batches and we will updates our weights 1000 times. With a batch size of 1000 we will update the weights 100 times in each epoch. With a batch size of 100000 we will update our weights 10 times in each epoch. As such the relation is n_weight_updates = data_set_size/batch_size for each epoch.

Q11: Our memory capacity of how much data we can process at once.

Q12: Generally speaking the batch size and learning rate should be tuned togheter, i.e when batch size is increased learning rate should also be increased and vice versa. 

# Part 14: Increasing the complexity

Lets try some different configurations of number of layers and number of nodes per layer.

Question 13: How many trainable parameters does the network with 4 dense layers with 50 nodes each have, compared to the initial network with 2 layers and 20 nodes per layer? Hint: use model.summary()

Q13: 4 layers 50 nodes have 12,351 parameters

2 layers 20 nodes have 2,301 parameters

### 4 layers, 20 nodes, class weights

In [12]:
# Setup some training parameters
batch_size = 10000
epochs = 20
input_shape = 92

# Build and train model
model3 = build_DNN(input_shape, n_layers = 4, n_nodes = 20, learning_rate = 0.1)

history3 = model3.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)

#model3.summary()

Epoch 1/20
54/54 [==============================] - 1s 12ms/step - loss: 0.7182 - accuracy: 0.6047 - val_loss: 0.6932 - val_accuracy: 0.4488
Epoch 2/20
54/54 [==============================] - 1s 10ms/step - loss: 0.6931 - accuracy: 0.5242 - val_loss: 0.6942 - val_accuracy: 0.1568
Epoch 3/20
54/54 [==============================] - 0s 9ms/step - loss: 0.6927 - accuracy: 0.5127 - val_loss: 0.6940 - val_accuracy: 0.1586
Epoch 4/20
54/54 [==============================] - 0s 8ms/step - loss: 0.6924 - accuracy: 0.5012 - val_loss: 0.6895 - val_accuracy: 0.8398
Epoch 5/20
54/54 [==============================] - 0s 9ms/step - loss: 0.6920 - accuracy: 0.7370 - val_loss: 0.6916 - val_accuracy: 0.8797
Epoch 6/20
54/54 [==============================] - 0s 8ms/step - loss: 0.6916 - accuracy: 0.7035 - val_loss: 0.6907 - val_accuracy: 0.8820
Epoch 7/20
54/54 [==============================] - 0s 8ms/step - loss: 0.6911 - accuracy: 0.7607 - val_loss: 0.6892 - val_accuracy: 0.8713
Epoch 8/20
54/54 [

In [ ]:
# Evaluate model on test data
score = model3.evaluate(Xtest, Ytest)

print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
plot_results(history3)

### 2 layers, 50 nodes, class weights

In [ ]:
# Setup some training parameters
batch_size = 10000
epochs = 20
input_shape = 92

# Build and train model
model4 = build_DNN(input_shape, n_layers = 2, n_nodes = 50, learning_rate = 0.1)

history4 = model4.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)

In [ ]:
# Evaluate model on test data
score = model4.evaluate(Xtest, Ytest)

print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
plot_results(history4)

### 4 layers, 50 nodes, class weights

In [ ]:
# Setup some training parameters
batch_size = 10000
epochs = 20
input_shape = 92

# Build and train model
model5 = build_DNN(input_shape, n_layers = 4, n_nodes = 50, learning_rate = 0.1)

history5 = model5.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)

In [ ]:
# Evaluate model on test data
score = model5.evaluate(Xtest, Ytest)

print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
plot_results(history5)

# Part 15: Batch normalization

Now add batch normalization after each dense layer in `build_DNN`. Remember to import BatchNormalization from keras.layers. 

See https://keras.io/layers/normalization/ for information about how to call the function.

Question 14: Why is batch normalization important when training deep networks?

Q14: When training deep networks it's important to have stable weights that doesn't become imbalanced with extreme values (or gradient that explode), as this could cause our network to only regard certain layers/activations. As such we use batch normalization to normalize the output of the activation functions.

In [13]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy

# Set seed from random number generator, for better comparisons
from numpy.random import seed
seed(123)

def build_DNN(input_shape, n_layers, n_nodes, act_fun='sigmoid', optimizer='sgd', learning_rate=0.01, 
              use_bn=False, use_dropout=False, use_custom_dropout=False):
    
    # Setup optimizer, depending on input parameter string
    if optimizer == 'sgd':
        opt = SGD(learning_rate = learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate = learning_rate)
        
    # Setup a sequential model
    model = Sequential()

    # Add layers to the model, using the input parameters of the build_DNN function
  

    # Add first layer, requires input shape
    model.add(Dense(units = n_nodes,
                    input_dim = input_shape, 
                    activation = act_fun))
    if use_bn:
        model.add(BatchNormalization())
    if use_dropout:
        model.add(Dropout(rate=0.5))
    if use_custom_dropout:
        model.add(myDropout(rate=0.5))
        
    # Add remaining layers, do not require input shape
    for i in range(n_layers-1):
        model.add(Dense(units = n_nodes,
                       activation = act_fun))
        if use_bn:
            model.add(BatchNormalization())
        if use_dropout:
            model.add(Dropout(rate=0.5))
        if use_custom_dropout:
            model.add(myDropout(rate=0.5))


    # Add final layer
    model.add(Dense(units=1, activation = act_fun)) # 1 node
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer = opt,
                  metrics=['accuracy'])
    
    return model

### 2 layers, 20 nodes, class weights, batch normalization

In [ ]:
# Setup some training parameters
batch_size = 10000
epochs = 20
input_shape = 92

# Build and train model
model6 = build_DNN(input_shape, n_layers = 2, n_nodes = 20, learning_rate = 0.1, use_bn=True)

history6 = model6.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)

In [ ]:
# Evaluate model on test data
score = model6.evaluate(Xtest, Ytest)

print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
plot_results(history6)

# Part 16: Activation function

Try changing the activation function in each layer from sigmoid to ReLU, write down the test accuracy.

Note: the last layer should still have a sigmoid activation function.

https://keras.io/api/layers/activations/

### 2 layers, 20 nodes, class weights, ReLU, no batch normalization

In [ ]:
# Setup some training parameters
batch_size = 10000
epochs = 20
input_shape = 92

# Build and train model
model7 = build_DNN(input_shape, n_layers = 2, n_nodes = 20, learning_rate = 0.1, use_bn=False, act_fun='relu')

history7 = model7.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)

In [ ]:
# Evaluate model on test data
score = model7.evaluate(Xtest, Ytest)

print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
plot_results(history7)

# Part 17: Optimizer

Try changing the optimizer from SGD to Adam (with learning rate 0.1 as before). Remember to import the Adam optimizer from keras.optimizers. 

https://keras.io/optimizers/


### 2 layers, 20 nodes, class weights, Adam optimizer, no batch normalization, sigmoid activations

In [ ]:
# Setup some training parameters
batch_size = 10000
epochs = 20
input_shape = 92

# Build and train model
model8 = build_DNN(input_shape, n_layers = 2, n_nodes = 20, learning_rate = 0.1, 
                   use_bn=False, act_fun='sigmoid', optimizer='adam')

history8 = model7.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)

In [ ]:
# Evaluate model on test data
score = model8.evaluate(Xtest, Ytest)

print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
plot_results(history8)

# Part 18: Dropout regularization

Dropout is a type of regularization that can improve accuracy for validation and test data. 

Add a Dropout layer after each Dense layer (but not after the final dense layer) in `build_DNN`, with a dropout probability of 50%. Remember to first import the Dropout layer from keras.layers

See https://keras.io/api/layers/regularization_layers/dropout/ for how the Dropout layer works.

---

Question 15: How does the validation accuracy change when adding dropout?

Question 16: How does the test accuracy change when adding dropout?

Q15:  With dropout val_accuracy: 0.8840.
        W/o dropout: val_accuracy: 0.9042. It decreases.
        
        
Q16: With dropout test accuracy: 0.8819
        W/o dropout: 0.9022. It decreases.

### 2 layers, 20 nodes, class weights, dropout, SGD optimizer, no batch normalization, sigmoid activations

In [20]:
# Setup some training parameters
batch_size = 10000
epochs = 20
input_shape = 92

# Build and train model
model9 = build_DNN(input_shape, n_layers = 2, n_nodes = 20, learning_rate = 0.1, 
                   use_dropout = True, act_fun='sigmoid', optimizer='sgd')

history9 = model9.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)

Epoch 1/20
54/54 [==============================] - 1s 14ms/step - loss: 0.6927 - accuracy: 0.5835 - val_loss: 0.6127 - val_accuracy: 0.8822
Epoch 2/20
54/54 [==============================] - 1s 12ms/step - loss: 0.6111 - accuracy: 0.6684 - val_loss: 0.5269 - val_accuracy: 0.8821
Epoch 3/20
54/54 [==============================] - 1s 10ms/step - loss: 0.5372 - accuracy: 0.7462 - val_loss: 0.4313 - val_accuracy: 0.8821
Epoch 4/20
54/54 [==============================] - 1s 10ms/step - loss: 0.4671 - accuracy: 0.7953 - val_loss: 0.3586 - val_accuracy: 0.8820
Epoch 5/20
54/54 [==============================] - 1s 9ms/step - loss: 0.4148 - accuracy: 0.8236 - val_loss: 0.3201 - val_accuracy: 0.8820
Epoch 6/20
54/54 [==============================] - 1s 10ms/step - loss: 0.3798 - accuracy: 0.8398 - val_loss: 0.3012 - val_accuracy: 0.8819
Epoch 7/20
54/54 [==============================] - 0s 8ms/step - loss: 0.3546 - accuracy: 0.8507 - val_loss: 0.2903 - val_accuracy: 0.8820
Epoch 8/20
54/5

In [ ]:
# Evaluate model on test data
score = model9.evaluate(Xtest, Ytest)

print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
plot_results(history9)

# Part 19: Improving performance

Spend some time (30 - 90 minutes) playing with the network architecture (number of layers, number of nodes per layer, activation function) and other hyper parameters (optimizer, learning rate, batch size, number of epochs, degree of regularization). For example, try a much deeper network. How much does the training time increase for a network with 10 layers?

Question 17: How high classification accuracy can you achieve for the test data? What is your best configuration?

The deeper the network, the more time it takes to train the network. 
After playing around with the hyperparameters the settings below resulted in the highest classification accuracy for the test data. 

Q17: 89.72%

In [26]:
# Find your best configuration for the DNN
batch_size = 10000
epochs = 25
input_shape = 92

# Build and train DNN
model10 = build_DNN(input_shape, n_layers = 3, n_nodes = 50, learning_rate = 0.3, 
                   use_dropout = True, act_fun='sigmoid', optimizer='sgd')

history10 = model10.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)

Epoch 1/25
54/54 [==============================] - 1s 21ms/step - loss: 0.7146 - accuracy: 0.5225 - val_loss: 0.6965 - val_accuracy: 0.1596
Epoch 2/25
54/54 [==============================] - 1s 18ms/step - loss: 0.6846 - accuracy: 0.5546 - val_loss: 0.6665 - val_accuracy: 0.8885
Epoch 3/25
54/54 [==============================] - 1s 18ms/step - loss: 0.6591 - accuracy: 0.6109 - val_loss: 0.5984 - val_accuracy: 0.8863
Epoch 4/25
54/54 [==============================] - 1s 18ms/step - loss: 0.5608 - accuracy: 0.7214 - val_loss: 0.3878 - val_accuracy: 0.8824
Epoch 5/25
54/54 [==============================] - 1s 19ms/step - loss: 0.3996 - accuracy: 0.8289 - val_loss: 0.2968 - val_accuracy: 0.8818
Epoch 6/25
54/54 [==============================] - 1s 19ms/step - loss: 0.3196 - accuracy: 0.8650 - val_loss: 0.2958 - val_accuracy: 0.8818
Epoch 7/25
54/54 [==============================] - 1s 19ms/step - loss: 0.2917 - accuracy: 0.8739 - val_loss: 0.2919 - val_accuracy: 0.8818
Epoch 8/25
54

In [ ]:
# Evaluate DNN on test data
score = model10.evaluate(Xtest, Ytest)
print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

# Part 20: Dropout uncertainty

Dropout can also be used during testing, to obtain an estimate of the model uncertainty. Since dropout will randomly remove connections, the network will produce different results every time the same (test) data is put into the network. This technique is called Monte Carlo dropout. For more information, see this paper http://proceedings.mlr.press/v48/gal16.pdf

To achieve this, we need to redefine the Keras Dropout call by running the cell below, and use 'myDropout' in each call to Dropout, in the cell that defines the DNN. The `build_DNN` function takes two boolean arguments, use_dropout and use_custom_dropout, add a standard Dropout layer if use_dropout is true, add a myDropout layer if use_custom_dropout is true.

Run the same test data through the trained network 100 times, with dropout turned on. 

Question 18: What is the mean and the standard deviation of the test accuracy?

In [27]:
import keras.backend as K
import keras

class myDropout(keras.layers.Dropout):
    """Applies Dropout to the input.
    Dropout consists in randomly setting
    a fraction `rate` of input units to 0 at each update during training time,
    which helps prevent overfitting.
    # Arguments
        rate: float between 0 and 1. Fraction of the input units to drop.
        noise_shape: 1D integer tensor representing the shape of the
            binary dropout mask that will be multiplied with the input.
            For instance, if your inputs have shape
            `(batch_size, timesteps, features)` and
            you want the dropout mask to be the same for all timesteps,
            you can use `noise_shape=(batch_size, 1, features)`.
        seed: A Python integer to use as random seed.
    # References
        - [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](
           http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)
    """
    def __init__(self, rate, training=True, noise_shape=None, seed=None, **kwargs):
        super(myDropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

In [28]:
# Your best training parameters
batch_size = 10000
epochs = 25
input_shape = 92


# Build and train model
model11 = build_DNN(input_shape, n_layers = 3, n_nodes = 50, learning_rate = 0.3, 
                   use_custom_dropout = True, act_fun='sigmoid', optimizer='sgd')

history11 = model11.fit(Xtrain, Ytrain, validation_data = (Xval, Yval), epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)


Epoch 1/25
54/54 [==============================] - 2s 21ms/step - loss: 0.7080 - accuracy: 0.5345 - val_loss: 0.6894 - val_accuracy: 0.5394
Epoch 2/25
54/54 [==============================] - 1s 18ms/step - loss: 0.6667 - accuracy: 0.5941 - val_loss: 0.6636 - val_accuracy: 0.5911
Epoch 3/25
54/54 [==============================] - 1s 19ms/step - loss: 0.5849 - accuracy: 0.6917 - val_loss: 0.5059 - val_accuracy: 0.7610
Epoch 4/25
54/54 [==============================] - 1s 18ms/step - loss: 0.4194 - accuracy: 0.8072 - val_loss: 0.3816 - val_accuracy: 0.8422
Epoch 5/25
54/54 [==============================] - 1s 19ms/step - loss: 0.3223 - accuracy: 0.8564 - val_loss: 0.3461 - val_accuracy: 0.8656
Epoch 6/25
54/54 [==============================] - 1s 19ms/step - loss: 0.2862 - accuracy: 0.8699 - val_loss: 0.3148 - val_accuracy: 0.8754
Epoch 7/25
54/54 [==============================] - 1s 20ms/step - loss: 0.2678 - accuracy: 0.8752 - val_loss: 0.3030 - val_accuracy: 0.8789
Epoch 8/25
54

In [ ]:
# Run this cell a few times to evalute the model on test data, 
# if you get slightly different test accuracy every time, Dropout during testing is working

# Evaluate model on test data
score = model11.evaluate(Xtest, Ytest)
                       
print('Test accuracy: %.4f' % score[1])

for _ in range(0, 20):
    score = model11.evaluate(Xtest, Ytest)
    print('Test accuracy: %.4f' % score[1])

In [ ]:
# Run the testing 100 times, and save the accuracies in an array
accuracies = np.empty(shape=100)
for i in range(0, 100):
    score = model11.evaluate(Xtest, Ytest)
    accuracies[i] = score[1]
    
# Calculate and print mean and std of accuracies
print(np.mean(accuracies))
print(np.std(accuracies))


# Part 21: Cross validation uncertainty

Cross validation (CV) is often used to evaluate a model, by training and testing using different subsets of the data it is possible to get the uncertainty as the standard deviation over folds. We here use a help function from scikit-learn to setup the CV, see https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html . Use 10 folds with shuffling, random state 1234. 

Note: We here assume that you have found the best hyper parameters, so here the data are only split into training and testing, no validation.

---

Question 19: What is the mean and the standard deviation of the test accuracy?

Question 20: What is the main advantage of dropout compared to CV for estimating test uncertainty? The difference may not be so large in this notebook, but imagine that you have a network that takes 24 hours to train.

In [32]:
from sklearn.model_selection import StratifiedKFold

# Define 10-fold cross validation
skf = StratifiedKFold(n_splits=10, random_state=1234, shuffle=True)

# np array for keeping score
np_score = np.empty(shape=10)
# Loop over cross validation folds
ctr = 0
for train_split, test_split in skf.split(X, Y):    
    # Calculate class weights for current split
    
    class_weights = class_weight.compute_class_weight(class_weight = 'balanced', 
                                                      classes = np.unique(Y[train_split]), y = Y[train_split])
    
    class_weights = {0: class_weights[0],
                 1: class_weights[1]}
    
    # Rebuild the DNN model, to not continue training on the previously trained model
    rebuild_model = build_DNN(input_shape, n_layers = 3, n_nodes = 50, learning_rate = 0.3, 
                   use_dropout = False, act_fun='sigmoid', optimizer='sgd')
    
    # Fit the model with training set and class weights for this fold
    fit_model = rebuild_model.fit(X[train_split], Y[train_split], validation_data = False, epochs=epochs, batch_size = batch_size,
                     class_weight = class_weights)
    
    # Evaluate the model using the test set for this fold
    score = rebuild_model.evaluate(X[test_split], Y[test_split])
    
    # Save the test accuracy in an array

    np_score[ctr]= score[1]
    
    ctr += 1
    
# Calculate and print mean and std of accuracies
print(np.mean(np_score))
print(np.std(np_score))


Epoch 1/25
69/69 [==============================] - 1s 10ms/step - loss: 0.6403 - accuracy: 0.8607
Epoch 2/25
69/69 [==============================] - 1s 10ms/step - loss: 0.3137 - accuracy: 0.8862
Epoch 3/25
69/69 [==============================] - 1s 10ms/step - loss: 0.2109 - accuracy: 0.8919
Epoch 4/25
69/69 [==============================] - 1s 9ms/step - loss: 0.1989 - accuracy: 0.8965
Epoch 5/25
69/69 [==============================] - 1s 10ms/step - loss: 0.1925 - accuracy: 0.8979
Epoch 6/25
69/69 [==============================] - 1s 10ms/step - loss: 0.1880 - accuracy: 0.8993
Epoch 7/25
69/69 [==============================] - 1s 10ms/step - loss: 0.1846 - accuracy: 0.9016
Epoch 8/25
69/69 [==============================] - 1s 9ms/step - loss: 0.1819 - accuracy: 0.9046
Epoch 9/25
69/69 [==============================] - 1s 10ms/step - loss: 0.1797 - accuracy: 0.9070
Epoch 10/25
69/69 [==============================] - 1s 10ms/step - loss: 0.1779 - accuracy: 0.9085
Epoch 11/25

69/69 [==============================] - 1s 11ms/step - loss: 0.1805 - accuracy: 0.9072
Epoch 10/25
69/69 [==============================] - 1s 12ms/step - loss: 0.1785 - accuracy: 0.9090
Epoch 11/25
69/69 [==============================] - 1s 12ms/step - loss: 0.1768 - accuracy: 0.9099
Epoch 12/25
69/69 [==============================] - 1s 13ms/step - loss: 0.1755 - accuracy: 0.9104
Epoch 13/25
69/69 [==============================] - 1s 12ms/step - loss: 0.1744 - accuracy: 0.9107
Epoch 14/25
69/69 [==============================] - 1s 12ms/step - loss: 0.1734 - accuracy: 0.9109
Epoch 15/25
69/69 [==============================] - 1s 12ms/step - loss: 0.1725 - accuracy: 0.9111
Epoch 16/25
69/69 [==============================] - 1s 12ms/step - loss: 0.1718 - accuracy: 0.9121
Epoch 17/25
69/69 [==============================] - 1s 12ms/step - loss: 0.1711 - accuracy: 0.9133
Epoch 18/25
69/69 [==============================] - 1s 12ms/step - loss: 0.1704 - accuracy: 0.9138
Epoch 19/25


Q20: Running dropout requires us to run the model with dropout once, but using CV we will run the model k-folds time which in some instances can be cumbersome/take too much time.

# Part 22: DNN regression

A similar DNN can be used for regression, instead of classification.

Question 21: How would you change the DNN in order to use it for regression instead?

Q21: 
We would change to one output neuron/node and change the loss function to something like mean squared error.

## Report

Send in this jupyter notebook, with answers to all questions.